In [1]:
%load_ext blackcellmagic

In [2]:
%%black

import pandas as pd

# define functions for processing featureCount and experimental design


def process_keytable(
    filepath_keytable,
    filepath_output,
    sep_keytable="\t",
    GCS=False,
    print_debug=False,
):
    keytable = pd.read_csv(filepath_keytable, sep=sep_keytable)
    ##Need to generalize
    keytable["SampleID-Lab"] = keytable["Description"].str[-10:]
    keytable.rename(columns={"Sample_ID": "SampleID"}, inplace=True)
    if print_debug:
        print(keytable["SampleID"][0])
    keytable.to_csv(path_or_buf=filepath_output, sep="\t")
    return keytable


def get_ID_dict(keytable_df):
    ID_dict = {
        keytable_df["SampleID"][row]: keytable_df["SampleID-Lab"][row]
        for row in keytable_df.index
    }
    return ID_dict


def process_featureCounts(
    filepath_featureCount,
    filepath_featureCount_relabeled,
    ID_dict,
    sep_featureCount="\t",
    GCS=False,
    print_debug=False,
):
    featureCount = pd.read_csv(filepath_featureCount, sep=sep_featureCount)
    featureCount = featureCount.reindex(sorted(featureCount.columns), axis=1)
    fc_columns = featureCount.columns
    sample_names = fc_columns[0:-2]
    if print_debug:
        print(sample_names)
    file_dict = {name: name[0:8] for name in sample_names}
    featureCount.rename(columns=file_dict, inplace=True)
    featureCount.rename(columns=ID_dict, inplace=True)

    # rearrange columns to put geneid and gene name at front
    columns = featureCount.columns.tolist()
    columns = columns[-2:] + columns[:-2]
    featureCount = featureCount[columns]

    # write relabeled featureCounts to tsv file
    featureCount.to_csv(path_or_buf=filepath_featureCount_relabeled, sep="\t")

    return featureCount


def generate_experiment_design_table(
    keytable,
    filepath_exp_design,
    keytable_column="SampleID-Lab",
    info_list=[
        "Cell Line",
        "Inhibition Status",
        "CRISPR",
        "MRTX",
        "BI",
        "SHP2i",
        "Time Point",
        "Population",
    ],
):
    exp_design = pd.DataFrame()
    exp_design[keytable_column] = keytable[keytable_column]
    # create sample info columns
    for x in info_list:
        exp_design[x] = exp_design[keytable_column].apply(
            lambda row: extract_info_exp_design(row, x)
        )
    exp_design.to_csv(path_or_buf=filepath_exp_design, sep="\t")
    return exp_design


def extract_info_exp_design(ID, info):
    """Read the lab ID to extract info about the sample."""
    if info == "Cell Line":
        return ID[2:4]
    if info == "CRISPR":
        return ID[4:6]
    if len(ID) != 10:
        if info == "Inhibition Status":
            return ID[4:6] + "XX"
        if info == "MRTX" or info == "BI" or info == "SHP2i":
            return False
        if info == "Time Point":
            return 0
        if info == "Population":
            return "A"
    else:
        if info == "Inhibition Status":
            return ID[4:8]
        if info == "MRTX":
            if "M" in ID[6:8]:
                return True
            else:
                return False
        if info == "BI":
            if "B" in ID[6:8]:
                return True
            else:
                return False
        if info == "SHP2i":
            if "S" in ID[6:8]:
                return True
            else:
                return False
        if info == "Time Point":
            if int(ID[8:10]) <= 4 and ID[6:8] == "XX":
                return 0
            elif int(ID[8:10]) <= 4:
                return 6
            else:
                return 72
        if info == "Population":
            if int(ID[8:10]) in [1, 2, 5, 6]:
                return "A"
            else:
                return "B"


def get_gene_list(filepath_gene_list):
    # get list of genes from text file
    with open(filepath_gene_list) as f:
        genes = f.read().splitlines()
    while "" in genes:
        genes.remove("")
    return genes


def make_featureCount_genelist(featureCount, gene_list, filepath):
    # check that genes are named in featureCount
    good_genes = []
    print(
        "These genes are not listed in featureCount and will not be included in the analysis:"
    )
    for gene in gene_list:
        if gene in featureCount["gene_name"].tolist():
            good_genes.append(gene)
        else:
            print(gene)
    print("All other genes in the list will be included.")
    ###ADD PRINT NUMBER OF GENES IN LIST
    # make featureCount dataframe with existing genes
    featureCount_genes = featureCount[
        featureCount["gene_name"].isin(good_genes)
    ]
    featureCount_genes.to_csv(path_or_buf=filepath, sep="\t")
    return featureCount_genes

In [3]:
#import H23 featureCounts and keytable
keytable_H23 = process_keytable("data_keytable.2020.10.27.csv", "20210113_H23_keytable.tsv")
ID_dict_H23 = get_ID_dict(keytable)
featureCount_H23 = process_featureCounts("runs_20210113-results_featureCounts_merged_gene_counts.txt", "20210113_featureCount_H23.tsv", ID_dict_H23, , sep_featureCount = ",")
exp_design_H23 = generate_experiment_design_table(keytable, "20210103_experiment_design_H23.tsv")


InvalidInput: Cannot parse: 4:149: featureCount_H23 = process_featureCounts("runs_20210113-results_featureCounts_merged_gene_counts.txt", "20210113_featureCount_H23.tsv", ID_dict_H23, , sep_featureCount = ",")

In [15]:
#import H358 featureCounts and keytable
keytable_H358 = process_keytable("H358_keytable.txt", "20210209_H358_keytable.tsv")
ID_dict_H358 = get_ID_dict(keytable_H358)
featureCount_H358 = process_featureCounts("runs_20210209-results_featureCounts_merged_gene_counts.txt", "20210209_featureCount_H358.tsv", ID_dict_H358)
exp_design_H358 = generate_experiment_design_table(keytable_H358, "20210209_experiment_design_H358.tsv")

In [22]:
#H358 ALDH
gene_set_name = "ALDH"
cell_line = "H358"
date = "20210209"
file_output = date + "_featureCounts_" + cell_line + "-" + gene_set_name + ".tsv"
ALDH_genes = get_gene_list("ALDH genes.txt")
make_featureCount_genelist(featureCount_H358, ALDH_genes, file_output)

These genes are not listed in featureCount and will not be included in the analysis:
All other genes in the list will be included.


,Geneid,gene_name,ES58NTXX01,ES58NTXX02,ES58NTXX03,ES58NTXX04,ES58S1XX01,ES58S1XX02,ES58S1XX03,ES58S1XX04,...,ES58S1MX03,ES58S1MX04,ES58S2MX01,ES58S2MX02,ES58S2MX03,ES58S2MX04,ES58S2MB01,ES58S2MB02,ES58S2MB03,ES58S2MB04
604,ENSG00000159423,ALDH4A1,357,379,272,345,393,371,362,310,...,341,397,324,341,271,292,314,386,312,262
3722,ENSG00000143149,ALDH9A1,3629,3569,2764,4039,3977,3499,3227,3344,...,3741,3766,3367,3567,3530,3745,4060,4269,3870,3607
6974,ENSG00000059573,ALDH18A1,1943,1848,1458,2165,2218,1978,1995,2075,...,2510,2414,2419,2458,2394,2457,2246,2349,2104,2006
9452,ENSG00000132746,ALDH3B2,608,744,898,1143,985,732,1161,1117,...,476,564,417,482,833,992,558,641,982,992
9472,ENSG00000006534,ALDH3B1,3905,3728,2610,3080,3704,3213,3529,3355,...,2315,2287,2173,2440,1981,2138,3118,3055,2747,2545
13014,ENSG00000136010,ALDH1L2,44,40,58,130,53,56,57,61,...,75,51,35,54,107,80,85,63,101,117
13156,ENSG00000111275,ALDH2,90,69,99,164,92,60,75,92,...,74,70,52,75,84,91,103,78,76,89
16170,ENSG00000119711,ALDH6A1,3801,3272,2326,3320,3630,3209,3096,3112,...,3449,2863,2604,2903,2650,2685,3528,3334,2770,2599
18107,ENSG00000128918,ALDH1A2,44,31,38,42,20,29,15,36,...,14,14,21,21,23,28,8,5,20,31
19154,ENSG00000184254,ALDH1A3,2793,2324,1729,2479,2829,2283,2520,2485,...,7345,7432,6376,5826,6609,7331,8317,8710,8585,7415


In [23]:
#H358 stem cell associated factors
gene_set_name = "Stem_cell_assoc"
cell_line = "H358"
date = "20210209"
file_output = date + "_featureCounts_" + cell_line + "-" + gene_set_name + ".tsv"
SCA_genes = get_gene_list("Stem cell associated factors.txt")
make_featureCount_genelist(featureCount_H358, SCA_genes, file_output)

These genes are not listed in featureCount and will not be included in the analysis:
CTNNB
TCF1
OCT4
All other genes in the list will be included.


,Geneid,gene_name,ES58NTXX01,ES58NTXX02,ES58NTXX03,ES58NTXX04,ES58S1XX01,ES58S1XX02,ES58S1XX03,ES58S1XX04,...,ES58S1MX03,ES58S1MX04,ES58S2MX01,ES58S2MX02,ES58S2MX03,ES58S2MX04,ES58S2MB01,ES58S2MB02,ES58S2MB03,ES58S2MB04
1410,ENSG00000117425,PTCH2,10,5,5,3,14,6,9,23,...,11,14,15,3,13,20,10,14,6,14
2590,ENSG00000134245,WNT2B,104,143,109,193,137,117,133,173,...,86,105,94,90,125,142,100,168,140,163
2758,ENSG00000134250,NOTCH2,9569,9719,8682,12161,12207,10148,12694,13453,...,9797,10284,9425,10674,11519,13141,11686,10900,11714,12151
4871,ENSG00000154342,WNT3A,78,67,39,56,59,49,49,44,...,38,44,36,29,33,41,41,65,26,50
5716,ENSG00000168283,BMI1,708,721,621,836,814,608,593,735,...,826,848,780,785,718,976,929,897,927,778
10082,ENSG00000184384,MAML2,2935,2921,2601,3464,3242,2885,3358,3615,...,2900,2479,2546,2762,2766,3288,3584,3327,3188,2821
11057,ENSG00000111704,NANOG,3,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,1,3,0
11822,ENSG00000169884,WNT10B,0,0,3,1,4,0,2,2,...,0,1,2,4,0,1,0,10,0,3
11824,ENSG00000125084,WNT1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11831,ENSG00000139549,DHH,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,2,1
